In [1]:
#  project of logistic regression to check accuracy on titanic dat 

In [52]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [53]:
df=pd.read_csv("D://pandas//titanic.csv")

In [54]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [55]:
df=df[['Survived','Sex','Pclass','Fare','Embarked','Age']]#selecting main columns



In [56]:
df.head()

,Survived,Sex,Pclass,Fare,Embarked,Age
0,0,male,3,7.8292,Q,34.5
1,1,female,3,7.0000,S,47.0
2,0,male,2,9.6875,Q,62.0
3,0,male,3,8.6625,S,27.0
4,1,female,3,12.2875,S,22.0


In [57]:
x=df.drop(columns=['Survived'])
y=df['Survived']

In [11]:
x.isnull().sum()

Sex          0
Pclass       0
Fare         1
Embarked     0
Age         86
dtype: int64

In [12]:
x.shape

(418, 5)

In [13]:
x.dropna(how='all',inplace=True)

In [14]:
x.shape

(418, 5)

In [15]:
x.describe()#checking min max values to get idea of outliers we can also use boxplot 

,Pclass,Fare,Age
count,418.000000,417.000000,332.000000
mean,2.265550,35.627188,30.272590
std,0.841838,55.907576,14.181209
min,1.000000,0.000000,0.170000
25%,1.000000,7.895800,21.000000
50%,3.000000,14.454200,27.000000
75%,3.000000,31.500000,39.000000
max,3.000000,512.329200,76.000000


In [16]:
temp=x.copy()#copy dataset in temp because we want fill null values in temp because if we directly fill then data leakage in pipeloine
#occured and also we can detect outliers next so we can remove in temp the null values

In [17]:
temp.head(2)

,Sex,Pclass,Fare,Embarked,Age
0,male,3,7.8292,Q,34.5
1,female,3,7.0000,S,47.0


In [18]:
# temp['Age'].fillna(temp['Age'].median(), inplace=True)
# temp['Fare'].fillna(temp['Fare'].median(), inplace=True)

In [19]:
temp.isnull().sum()

Sex          0
Pclass       0
Fare         1
Embarked     0
Age         86
dtype: int64

In [20]:
def removeoutlier(data,column):
    per75=data[column].quantile(0.75)
    per25=data[column].quantile(0.25)
    iqr=per75-per25
    lowerlimit=per25-1.5*iqr
    upperlimit=per75+1.5*iqr
    return data[(data[column]>=lowerlimit) & (data[column]<=upperlimit)]

temp=removeoutlier(temp,'Age')
temp=removeoutlier(temp,'Fare')

In [21]:
temp.describe()#outliersa are rempoved 

,Pclass,Fare,Age
count,296.000000,296.000000,296.000000
mean,2.277027,23.155152,28.805743
std,0.792515,20.027635,13.261812
min,1.000000,0.000000,0.170000
25%,2.000000,7.925000,21.000000
50%,2.000000,13.881250,26.000000
75%,3.000000,27.937500,36.625000
max,3.000000,83.158300,64.000000


In [22]:
x.shape


(418, 5)

In [23]:
temp.shape

(296, 5)

In [24]:
y.shape

(418,)

In [25]:
x=x.loc[temp.index]#Let’s say out of 100 rows, 10 had outliers, so now temp has 90 rows.
#But your original X and y still have all 100 rows. to update we can use loc method and in this we pass parameter .index 
y=y.loc[temp.index]

In [26]:
x.shape


(296, 5)

In [27]:
x.isnull().sum()#outliers are removed but null value is in because we can use simpleimputer 

Sex         0
Pclass      0
Fare        0
Embarked    0
Age         0
dtype: int64

In [28]:
x.describe()

,Pclass,Fare,Age
count,296.000000,296.000000,296.000000
mean,2.277027,23.155152,28.805743
std,0.792515,20.027635,13.261812
min,1.000000,0.000000,0.170000
25%,2.000000,7.925000,21.000000
50%,2.000000,13.881250,26.000000
75%,3.000000,27.937500,36.625000
max,3.000000,83.158300,64.000000


In [29]:
#now split data because our data is clean no outliyers no null values we also drop our target column 

In [30]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [31]:
#and then we make pipelines for apply standarization encoding simple 

In [42]:
num_cols=x_train.select_dtypes(include=['int','float']).columns.tolist()
cat_cols=x_train.select_dtypes(include=['object','category']).columns.tolist()

In [43]:
pipe1=Pipeline(steps=[
    ('tmf1',SimpleImputer(strategy='median')),
    ('tnf2',StandardScaler())
])

pipe2=Pipeline(steps=[
    ('tnf1',SimpleImputer(strategy='most_frequent')),
    ('tnf2',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor=ColumnTransformer(transformers=[
    ('first',pipe1,num_cols),
    ('second',pipe2,cat_cols)
])

In [44]:
finalpipe=Pipeline(steps=[
    ('pre',preprocessor),
    ('model',LogisticRegression())
])

In [45]:
finalpipe.fit(x_train,y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('first',
                                                  Pipeline(steps=[('tmf1',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('tnf2',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Fare', 'Age']),
                                                 ('second',
                                                  Pipeline(steps=[('tnf1',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('tnf2',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('model', LogisticRegression())])

In [46]:
finalpipe.score(x_train,y_train)

1.0

In [47]:
y_predic=finalpipe.predict(x_test)

In [48]:
accuracy_score(y_test,y_predic)

1.0

In [49]:
x_train.columns

Index(['Sex', 'Pclass', 'Fare', 'Embarked', 'Age'], dtype='object')

In [50]:
y_test.value_counts()

Survived
0    45
1    15
Name: count, dtype: int64

In [51]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(finalpipe, x, y, cv=5)
print("Cross-validation scores:", scores)
print("Mean CV Accuracy:", scores.mean())

Cross-validation scores: [1. 1. 1. 1. 1.]
Mean CV Accuracy: 1.0
